실제 파이썬 객체와 동일하게 동작하는 사용자 정의 클래스를 만들어보자. 여러 파이썬 객체에서 흔히 볼 수 있는 다음 지 특별 메서드를 구현하도록 한다.

* `repr()`, `bytes()` 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
* 클래스 메서드로 대안 생성자 구현
* `format()` 내장 함수와 `str.format()` 메서드에서 사용하는 포맷 언어 확장
* 읽기 전용 접근만 허용하는 속성 제공
* 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
* `__slots__`를 이용해서 메모리 절약하기
* 언제, 어떻게 `@classmethod`와 `@staticmethod` 데커레이터를 사용할 것인가
* 파이썬에서 비공개 및 보호된 속성: 사용법, 관례, 한계

# 9.1 객체 표현

* `repr()` : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환
* `str()` : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환

이들을 지원하려면 `__repr__()`과 `__str__()` 특별 메서드를 구현해야 한다. 그외에 `__bytes__()`와 `__format__()` 도 있다.

# 9.2 벡터 클래스의 부활

이 절부터는 `Vector2d` 클래스를 만들어서 계속 확장해나간다.

In [31]:
# vector2d_v0.py 
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [23]:
v1 = Vector2d(3,4)
v1

Vector2d(3.0, 4.0)

In [24]:
print(v1.x, v1.y) 

3.0 4.0


In [25]:
x,y=v1 # __iter__ 의 영향
print(x,y)

3.0 4.0


In [26]:
v1_clone = eval(repr(v1))
v1_clone, v1 == v1_clone # __eq__

(Vector2d(3.0, 4.0), True)

In [27]:
print(v1), repr(v1) # __str__, __repr__

(3.0, 4.0)


(None, 'Vector2d(3.0, 4.0)')

In [28]:
octets = bytes(v1) # __byte__
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [29]:
abs(v1), bool(v1), bool(Vector2d(0,0)) # __abs__, __bool__

(5.0, True, False)

# 9.3 대안 생성자

`Vector2d`를 `bytes`로 변환하는 메서드가 있으니 역으로 `bytes`를 `Vector2d`로 변환하는 메서드가 필요하다. 이를 추가해보자.

In [12]:
# vector2d_v1.py 
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [47]:
v2 = Vector2d.frombytes(b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@')
v2

Vector2d(3.0, 4.0)

In [49]:
v1 == v2 # 따라서 바이트코드로 정의해도 동일한 결과를 얻을 수 있다.

True

# 9.4 `@classmethod`와 `@staticmethod`

`@classmethod`는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의하며 클래스 자체를 첫번째 인수로 받게 만든다. `frombytes()` 같은 대안 생성자를 구현하기 위해 사용된다. 위의 예제에서는 마지막 문장에서 `cls(*memv)`를 호출해서 객체를 생성한다.

`@staticmethod` 데커레이터는 메서드가 특별한 첫번째 인수를 받지 않도록 메서드를 변경한다. 굳이 쓸 필요 없다...

In [30]:
class Demo:
    @classmethod
    def klassmeth(*args): # *args 나 아예 인수를 넣지 않는 경우는 자동으로 파이썬에서 cls를 첫번쨰 인수로 추가하는듯
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [31]:
Demo.klassmeth()

(__main__.Demo,)

In [32]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [7]:
Demo.statmeth()

()

In [8]:
Demo.statmeth('spam')

('spam',)

# 9.5 포맷된 출력

`format()`과 `str.format()` 메서드는 실제 포맷 작업을 `__format__(format_spec)` 메서드에 위임한다. `format_spec`은 포맷 명시자로서, 다음 중 하나의 방법으로 지정한다.

* `format(my_obj, format_spec)`의 두번째 인수
* `str.format()`에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 안에서 콜론 뒤의 문자열

In [50]:
brl = 1/2.43 # 브라질 레알을 미국 달러로 바꾸는 환율
brl, format(brl, '0.4f')

(0.4115226337448559, '0.4115')

In [51]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

포맷 명시자에 사용된 표기법은 [여기](https://docs.python.org/ko/3/library/string.html#formatspec)에서 확인할 수 있다. 또한 `str.format()` 메서드에 사용되는 `{:}` 형태의 표기법에 대해서는 [여기](https://docs.python.org/ko/3/library/string.html#formatstrings)를 참조하자. 여기서 `!s` 는 값에 `str()` 을 호출하고, `!r` 은 값에 `repr()` 을 호출하고, `!a` 는 값에 `ascii()` 를 호출한다. 또한 `int`형 자료의 경우 이진수를 나타내는 `b`, 16진수를 나타내는 `x`, `float`을 나타내는 `f`, 백분율을 나타내는 `%` 코드를 지원한다.

In [54]:
format(42,'b'), format(2/3, '.1%')

('101010', '66.7%')

`datetime` 모듈의 클래스들은 이런 형태를 지원한다.

In [55]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'20:39:18'

In [57]:
"It's now {:%I:%M %p}".format(now) # %I 는 12시간 기준인듯

"It's now 08:39 PM"

클래스에서 `__format__()` 메서드를 정의하지 않으면 `object`에서 상속받은 메서드가 `str(my_object)`를 반환한다.

In [58]:
v1 = Vector2d(3,4)
format(v1), str(v1), repr(v1), v1

('(3.0, 4.0)', '(3.0, 4.0)', 'Vector2d(3.0, 4.0)', Vector2d(3.0, 4.0))

그러나 포맷 명시자는 에러가 난다.

In [59]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

`Vector2d` 클래스에 포맷 간이 언어를 구현하면 이를 해결할 수 있다.

In [60]:
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)

In [64]:
v1 = Vector2d(3,4)
v1, str(v1), format(v1), format(v1,'.2f'), format(v1,'.3e'), 

(Vector2d(3.0, 4.0),
 '(3.0, 4.0)',
 '(3.0, 4.0)',
 '(3.00, 4.00)',
 '(3.000e+00, 4.000e+00)')

`Vector2d`의 간이 언어에 포맷 코드를 추가해보자. 포맷 명시자가 `'p'`로 끝나면 벡터를 극좌표 <r, $\theta$>로 표현하도록 해보자.

In [66]:
from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y) # 미리 실수로 변환하는 센스
        
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '<{}, {}>'            
        components = (format(c, fmt_spec) for c in coords)
        return '({}, {})'.format(*components)


In [69]:
format(Vector2d(1,1), 'p'), format(Vector2d(1,1), '.3ep'), format(Vector2d(1,1), '0.5fp')

('(1.4142135623730951, 0.7853981633974483)',
 '(1.414e+00, 7.854e-01)',
 '(1.41421, 0.78540)')

# 9.6 해시 가능한 `Vector2d`

지금까지 정의한 `Vector2d`는 해시할 수 없다.

In [71]:
v1 = Vector2d(3,4)
hash(v1)

TypeError: unhashable type: 'Vector2d'

In [72]:
set([v1])

TypeError: unhashable type: 'Vector2d'

해시 가능하게 하려면 `__hash__()` 메서드를 구현해야 한다. 그리고 `Vector2d` 객체를 불변형으로 만들어야 한다. 따라서 x와 y를 읽기 전용 속성으로 만들자.

In [80]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x) # 속성을 비공개로 만듦
        self.__y = float(y)

    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [85]:
v1 = Vector2d(3,4)
v1.x = 7

AttributeError: can't set attribute

`Vector2d`를 불변형으로 만들었으니 이제 `__hash__()` 메서드를 구현하자. `int`형을 반환해야 하며, `__hash__()` 특별 메서드 문서에서는 요소의 해시에 비트 단위 XOR 연산자(^)를 사용하는 것을 권장한다.

In [89]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x) # 속성을 비공개로 만듦
        self.__y = float(y)

    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
        
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [90]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [91]:
set([v1,v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

In [93]:
print(v1)

Vector2d(3.0, 4.0)


개선된 내용을 모두 반영한 `Vector2d` 클래스 코드는 다음과 같다.

In [94]:
# vector2d_v3.py

from array import array
import math

class Vector2d:
    typecode = 'd' # Vector2d와 bytes 간의 변환에 사용하는 클래스 속성
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y) # 미리 실수로 변환하는 센스
        
    @property # 게터 메서드를 나타냄
    def x(self):
        return self.__x        
        
    @property
    def y(self):
        return self.__y

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)        
    
    def __iter__(self): # 이걸 구현하면 x,y = my_vector 처럼 쓸 수 있다.
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '<{}, {}>'            
        components = (format(c, fmt_spec) for c in coords)
        return '({}, {})'.format(*components)

    @classmethod # 클래스 메서드
    def frombytes(cls, octets): # self 매개변수가 없고 대신 자신이 cls로 전달됨
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

# 9.7 파이썬에서의 비공개 속성과 보호된 속성

파이썬에는 비공개 변수를 생성할 수 있는 방법은 없지만 `__속성명`으로 지정하면 파이썬은 언더바와 클래스명을 변수명 앞에 붙여 저장한다. 예를 들어 `Vector2d` 클래스의 `__x`는 `_Vector2d__x`로 저장된다. 이런 기능을 `이름 장식(name mangling)`이라고 한다.

In [106]:
v1 = Vector2d(3,4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [107]:
v1._Vector2d__x = 4.1 # 강제로 수정할 수는 있음
v1

Vector2d(4.1, 4.0)

속성명 앞에 언더바 하나를 붙이더라도 파이썬 인터프리터가 처리하는 것은 없지만, 클래스 외부에서 그런 속성에 접근하지 않는 것은 파이썬 프로그래머 사이에 일종의 금기로 여겨진다. 또한 모듈에서 최상위 이름 앞에 언더바를 하나 붙이는 경우, `from my_mod import *`로 작성하면 해당 속성은 임포트하지 않는다.

# 9.8 `__slots__` 클래스 속성으로 공간 절약하기

기본적으로 파이썬은 객체 속성을 각 객체 안의 `__dict__`라는 딕셔너리 속성에 저장한다. 다만 딕셔너리는 메모리 부담이 크므로, 속성이 몇 개 없는 수백만 개의 객체를 다룬다면 `__slots__` 속성을 이용해서 객체 속성을 딕셔너리 대신 튜플에 저장하게 만들 수 있다.

In [108]:
class Vector2d:
    __slots__ = ('__x','__y')
    typecode = 'd'
    
    # 뒤는 생략

다만 다음 사항들을 주의해야한다.

* 상속된 클래스마다 `__slots__` 속성을 다시 정의해야 한다.
* `__dict__`를 `__slots__`에 추가하지 않는 한 객체는 `__slots__`에 나열된 속성만 가질 수 있다.
* `__weakref__`를 `__slots__`에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.

그러나 `__slots__`를 굳이 쓸 필요는 없으며... 대용량 데이터를 다룰 때는 `NumPy`나 `pandas`를 사용하자.

# 9.9 클래스 속성 오버라이드

클래스 속성을 객체 속성의 기본값으로 사용하는 것은 파이썬의 특징이다. `Vector2d` 객체는 자신의 `typecode` 속성을 가지고 있지 않으므로, `self.typecode`는 기본적으로 `Vector2d.typecode` 속성을 가져온다. 그러나 사본을  생성하므로 동일한 이름의 클래스 속성은 변경하지 않는다.

`Vector2d.typecode`의 기본값이 'd'이므로 객체를 생성할 때 `Vector2d`의 각 요소가 8바이트 배밀도 실수로 표현된다. 이를 변경하면 다음과 같이 된다.

In [109]:
from vector2d_v3 import Vector2d
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [110]:
len(dumpd)

17

In [111]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [112]:
len(dumpf)

9

In [114]:
Vector2d.typecode

'd'

이럴거면 그냥 `self.typecode = 'd'` 로 선언하는 거랑 뭐가 다른거지...

무튼 클래스 속성을 변경하고 싶을 때는 이를 상속하는 것이 일반적이다.

In [115]:
from vector2d_v3 import Vector2d
class ShortVector2d(Vector2d):
    typecode = 'f'

In [116]:
sv = ShortVector2d(1/11, 1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [117]:
len(bytes(sv))

9

상속하기 편하게 하기 위해서 이렇게 쓰는건가 싶기도 하고... (아니었으면 `def __init__(self)` 부터 귀찮게 정의했어야 하므로...)